In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [24]:
def release_token(accessToken):
    server = 'api.nrgstream.com'
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
print(default_pickle['accessToken'])
#release_token(default_pickle['accessToken'])

In [33]:
t = datetime(2022,1,1).date()
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (datetime.today().date(),monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (datetime.today().date(),daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:

with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': 'rLYSYf80aiQ1Aox5nx-ZY0Cn-OkumqMOxKIyzRDVhLsfiMtBcw36unHQrSpo8tCf1tBZOdszi5VelS-6D417V0sRoDwKMFOzq0E47PayTkfRmuC8x8ThaTtfQFLGdQQ6DInjz60hPsG93gsP6nKQin57arrNMCAF0V3oTCLdkoSzodfguLhM0OE1SJuhaKmV7M_1k4OuQt3UtwuVuv4PExY5DEb9Qb7NrP5AVvgD5kTLlwuh89UDQkgtft08xs3tiKJBQg8GCMInxZbmZ9n_yqZoOKohVXUXXaz6Lf3t44oe0-lOE0XUfZJxFvOcwWiyuqCV53WrY9UJuHNOKUruTzvBccs',
 'current_data': (datetime.datetime(2022, 9, 8, 0, 3, 28, 515020, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
     timeStamp  value         fuelType
  0 2022-09-08   1240             Coal
  0 2022-09-08      0        Dual Fuel
  0 2022-09-08      0   Energy Storage
  0 2022-09-08    244            Hydro
  0 2022-09-08   6044      Natural Gas
  0 2022-09-08    281  Biomass & Other
  0 2022-09-08      0            Solar
  0 2022-09-08   1053             Wind
  0 2022-09-08    137               BC
  0 2022-09-08    -36          Montana
  0 2022-09-08    -58     Saskatchewan),
 'daily_outage_dfs': [(datetime.d

In [69]:
with open('./monthly_df.pickle', 'rb') as handle:
    monthly_outage = pickle.load(handle)


In [4]:
default_pickle['current_data']

(datetime.datetime(2022, 9, 8, 0, 3, 28, 515020, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
    timeStamp  value         fuelType
 0 2022-09-08   1240             Coal
 0 2022-09-08      0        Dual Fuel
 0 2022-09-08      0   Energy Storage
 0 2022-09-08    244            Hydro
 0 2022-09-08   6044      Natural Gas
 0 2022-09-08    281  Biomass & Other
 0 2022-09-08      0            Solar
 0 2022-09-08   1053             Wind
 0 2022-09-08    137               BC
 0 2022-09-08    -36          Montana
 0 2022-09-08    -58     Saskatchewan)

In [ ]:
now = datetime.now(pytz.timezone('America/Edmonton'))
monthly_outage[monthly_outage['timeStamp'].dt.date<(now.date()+timedelta(days=90))]

In [64]:
[i for i in range(0,2001) if i%180==0]

[0, 180, 360, 540, 720, 900, 1080, 1260, 1440, 1620, 1800, 1980]

In [65]:
[i for i in range(0,2001) if i%300==0]

[0, 300, 600, 900, 1200, 1500, 1800]